<a href="https://colab.research.google.com/github/pedrocbt21/pedrocbt21/blob/main/SportsTreatment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial Functions

1. Connect to API-SPORTS
2. Get API_KEY from a secret file
3. Create Folder If Not Exists *Function*
4. Json to CSV *Function*
5. CSV to Panda DataFrame *Function*

## Ligação inicial à Api-Sports

**Documentação**: https://www.api-football.com/documentation-v3

In [ ]:
#Importar as bibliotecas necessárias
import http.client
import json
import pandas as pd
import time
from datetime import date
from datetime import datetime
from dateutil import parser

timeToSleep = 0

In [ ]:
#Criação e inicialização de variáveis
filePath = "/content/drive/MyDrive/B2F_PC/PEDRO/Mestrado/Tese/Project/Files"

In [ ]:
# Ir buscar a api key ao ficheiro privado
def get_keys(path):
    with open(path) as f:
        return json.load(f)

keys = get_keys("/content/drive/MyDrive/B2F_PC/PEDRO/Mestrado/Tese/Project/Files/.secret/api_football.json")
api_key = keys['api_key']

In [ ]:
url = "v3.football.api-sports.io"

headers = {
    'x-rapidapi-host': url,
    'x-rapidapi-key': api_key
    }

## GET API SPORTS

In [ ]:
def getApiSports(endPoint):
  conn = http.client.HTTPSConnection(url)
  #print(endPoint)
  conn.request("GET", "/" + endPoint, headers=headers)
  res = conn.getresponse()
  data = res.read()
  string_response = data.decode("utf-8")
  #print(string_response)
  json_obj = json.loads(string_response)
  time.sleep(timeToSleep) # time sleep colocado para não ultrapassar os limites por minuto da API em questão caso necessário (450 por minuto no momento)
  return json_obj

## Create Folder If Not Exists

In [ ]:
import os

def createFolder(finalPath):

  path = filePath + "/" + finalPath

  # Check whether the specified path exists or not
  isExist = os.path.exists(path)

  if not isExist:
  
    # Create a new directory because it does not exist 
    os.makedirs(path)
    #print("The new directory is created!")

## JSON to CSV

In [ ]:
def jsonToCsv(jsonKey, finalPath, filename, jsonObj):
  # Create dictionary of results for 'response' key ... /timezone
  json_dict = jsonObj[jsonKey]
  
  json_df = pd.DataFrame.from_dict(json_dict)
  # display(timezone_df) # caso seja para mostrar a informação é só descomentar esta linha

  #criação do caminho onde o csv file vai ser guardado
  jsoncsvfile = filePath + "/" + finalPath + "/" + filename + ".csv"

  #create folder if not exists 
  createFolder(finalPath)

  #save into csv file
  json_df.to_csv(jsoncsvfile, index=False)

  return finalPath

## Save Json

In [ ]:
def saveJson(finalPath, filename, jsonObj):
  #criação do caminho onde o csv file vai ser guardado
  jsonfile = filePath + "/" + finalPath + "/" + filename + ".json"

  #create folder if not exists 
  createFolder(finalPath)

  out_file = open(jsonfile, "w") 
    
  json.dump(jsonObj, out_file, indent = 6) 
    
  out_file.close() 

  return finalPath

## CSV --> Panda DataFrame

In [ ]:
def csvToDataFrame(fullPathFilename):
  df = pd.read_csv(fullPathFilename)
  return df
  # display(timezone_df)

## Import Json

In [ ]:
def importJson(fullPath):
  with open(f"{filePath}/{fullPath}") as f:
    data = json.load(f)
  return data

## TRY PARSE

### DATE

In [ ]:
def try_parse_date(dateFixture):
    try:
        return parser.parse(dateFixture).date()
    except:
        return print(date.today()+datetime.timedelta(days = 1)) 

# Timezone
https://www.api-football.com/documentation-v3#tag/Timezone/operation/get-timezone

In [ ]:
def getTimezonesToFile():
  jsonTimezone = getApiSports("timezone")
  #jsonToCsv("response", "timezone", "timezone", jsonTimezone)
  saveJson("timezone", "timezone", jsonTimezone)
  return jsonTimezone

# Countries

https://www.api-football.com/documentation-v3#tag/Countries

In [ ]:
def getCountriesToFile():
  jsonCountries = getApiSports("countries")
  #jsonToCsv("response", "countries", "countries", jsonCountries)
  saveJson("countries", "countries", jsonCountries)
  return jsonCountries

# Leagues

https://www.api-football.com/documentation-v3#tag/Leagues/operation/get-leagues

https://www.api-football.com/documentation-v3#tag/Leagues/operation/get-seasons

In [ ]:
def getLeaguesToFile():
  jsonLeagues = getApiSports("leagues")
  #jsonToCsv("response", "leagues", "leagues", jsonLeagues)
  saveJson("leagues", "leagues", jsonLeagues)
  return jsonLeagues

## SEASON


In [ ]:
def getSeasonsToFile():
  jsonSeasons = getApiSports("leagues/seasons")
  #jsonToCsv("response", "leagues/seasons", "seasons", jsonSeasons)
  saveJson("leagues/seasons", "seasons", jsonSeasons)
  return jsonSeasons

# Venue

https://www.api-football.com/documentation-v3#tag/Venues/operation/get-venues

Necessita do id do venue para os dados serem obtidos na API... Dados não relevantes para o nosso modelo.

O id do venue pode ser encontrado na API team Information

# Inicializar as Timezones/Countries/Leagues/Seasons

**Deve ser atualizado uma vez por dia**

In [ ]:
# É necessário 1 vez por dia
#jsonTimezones = getTimezonesToFile()
#jsonCountries = getCountriesToFile()
#jsonLeagues = getLeaguesToFile()
#jsonSeasons = getSeasonsToFile()

jsonTimezones = importJson("timezone/timezone.json")
jsonCountries = importJson("countries/countries.json")
jsonLeagues = importJson("leagues/leagues.json")
jsonSeasons = importJson("leagues/seasons/seasons.json")

# TEAMS

**Deve ser atualizado uma vez por dia na season atual**

https://www.api-football.com/documentation-v3#tag/Teams/operation/get-teams

https://www.api-football.com/documentation-v3#tag/Teams/operation/get-teams-statistics

https://www.api-football.com/documentation-v3#tag/Teams/operation/get-teams-seasons

https://www.api-football.com/documentation-v3#tag/Teams/operation/get-teams-countries

In [ ]:
def getTeamsInformation(season, leagueId):
  teamsInformationUrl = "teams?league=" + str(leagueId) + "&season=" + str(season)
  jsonLeagueSeason = getApiSports(teamsInformationUrl)
  path = "teams/league/" + str(leagueId) + "/" + str(season)
  filename = str(leagueId) + "_" + str(season) + "_teams_information"
  #jsonToCsv("response", path, filename, jsonLeagueSeason)
  saveJson(path, filename, jsonLeagueSeason)
  return jsonLeagueSeason

In [ ]:
def getAllSeasonLeagueInformation(jsonLeagues):
  # Escolher as ligas a serem estudadas
  leagueSeasons = jsonLeagues["response"]
  leaguesToSearch = [39,78,140,94,88,2,203,848,3,135,61]
  # 61 --> Ligue 1 --> France
  # 39 --> Premier League --> England
  # 78 --> Bundesliga --> Germany
  # 135 --> Serie A --> Italy
  # 88 --> Eredivisie --> Netherlands
  # 94 --> Primeira Liga --> Portugal
  # 140 --> La Liga --> Spain
  # 2 --> UEFA Champions League --> World
  # 203 --> Süper Lig --> Turkey
  # 3 --> UEFA Europa League --> World
  # 848 --> UEFA Europa Conference League --> World

  listOfSeasonLeague = []

  for x in leagueSeasons:
    if x["league"]["id"] in leaguesToSearch:
      for y in x["seasons"]:
        jsonLeagueSeason = "{\"league\":" + str(x["league"]["id"]) + " ,\"season\": " + str(y["year"]) + "}"
        listOfSeasonLeague.append(jsonLeagueSeason)
  
  return listOfSeasonLeague

In [ ]:
def getAllSeasonLeagueTeam(jsonLeagues):
  allSeasonLeagueInformation = getAllSeasonLeagueInformation(jsonLeagues)
  listOfSeasonLeagueTeams = []

  for x in allSeasonLeagueInformation:
    teamSeasonJson = json.loads(x)
    jsonLeagueSeasonTeamResp = getTeamsInformation(teamSeasonJson["season"],teamSeasonJson["league"])
    for team in jsonLeagueSeasonTeamResp["response"]:
      jsonLeagueSeasonTeam = "{\"league\":" + str(teamSeasonJson["league"]) + " ,\"season\": " + str(teamSeasonJson["season"]) + " ,\"team\": " + str(team["team"]["id"]) + "}"
      listOfSeasonLeagueTeams.append(str(jsonLeagueSeasonTeam))
  saveJson("teams", "allSeasonLeagueTeam", listOfSeasonLeagueTeams)
  return listOfSeasonLeagueTeams

In [ ]:
def getTeamsStatistics(season, leagueId, teamId):
  teamsStatisticsUrl = "teams/statistics?season=" + str(season) + "&team=" + str(teamId) + "&league=" + str(leagueId)
  jsonLeagueSeasonTeamStatistics = getApiSports(teamsStatisticsUrl)
  path = "teams/league/" + str(leagueId) + "/" + str(season)
  filename = str(leagueId) + "_" + str(season) + "_" + str(teamId) + "_team_statistics"
  saveJson(path, filename, jsonLeagueSeasonTeamStatistics)
  return jsonLeagueSeasonTeamStatistics

In [ ]:
def getAllSeasonLeagueTeamStatistics(listOfSeasonLeagueTeams):
  listOfSeasonLeagueTeamsStatistics = []
  
  for x in listOfSeasonLeagueTeams:
    teamSeasonLeagueJson = json.loads(x)
    jsonLeagueSeasonTeamStatisticsResp = getTeamsStatistics(teamSeasonLeagueJson["season"],teamSeasonLeagueJson["league"],teamSeasonLeagueJson["team"])
    listOfSeasonLeagueTeamsStatistics.append(str(jsonLeagueSeasonTeamStatisticsResp["response"]))

  return listOfSeasonLeagueTeamsStatistics

In [ ]:
#listOfSeasonLeagueTeams = getAllSeasonLeagueTeam(jsonLeagues)
#listOfSeasonLeagueTeamsStatistics = getAllSeasonLeagueTeamStatistics(listOfSeasonLeagueTeams)
listOfSeasonLeagueTeams = importJson("teams/allSeasonLeagueTeam.json")

## Team Squad Players

In [ ]:
def getTeamSquad(season, leagueId, teamId):
  teamSquadUrl = "players/squads?team=" + str(teamId)
  jsonLeagueSeasonTeamSquad = getApiSports(teamSquadUrl)
  path = "teams/league/" + str(leagueId) + "/" + str(season)
  filename = str(leagueId) + "_" + str(season) + "_" + str(teamId) + "_team_squad"
  saveJson(path, filename, jsonLeagueSeasonTeamSquad)
  return jsonLeagueSeasonTeamSquad

In [ ]:
def getAllLeagueTeamsSquad(listOfSeasonLeagueTeams):
  listOfSeasonLeagueTeamsSquad = []
  
  for x in listOfSeasonLeagueTeams:
    teamSeasonLeagueJson = json.loads(x)
    jsonLeagueSeasonTeamSquadResp = getTeamSquad(teamSeasonLeagueJson["season"],teamSeasonLeagueJson["league"],teamSeasonLeagueJson["team"])
    listOfSeasonLeagueTeamsSquad.append(str(jsonLeagueSeasonTeamSquadResp["response"]))

  return listOfSeasonLeagueTeamsSquad

In [ ]:
#listOfSeasonLeagueTeams = importJson("teams/allSeasonLeagueTeam.json")
#currentlySeason = 2022
#listOfSeasonLeagueTeamsFiltered = [x for x in listOfSeasonLeagueTeams if json.loads(x)["season"] == currentlySeason]
#listOfSeasonLeagueTeamsSquad = getAllLeagueTeamsSquad(listOfSeasonLeagueTeamsFiltered)

# Standings

**Deve ser atualizado uma vez por dia na season atual**

https://www.api-football.com/documentation-v3#tag/Standings

In [ ]:
def getSeasonStandings(season, leagueId):
  seasonStandingsUrl = "standings?season=" + str(season) + "&league=" + str(leagueId)
  jsonLeagueSeasonStandings = getApiSports(seasonStandingsUrl)
  path = "standing/league/" + str(leagueId) + "/" + str(season)
  filename = str(leagueId) + "_" + str(season) + "_season_standings"
  saveJson(path, filename, jsonLeagueSeasonStandings)
  return jsonLeagueSeasonStandings

In [ ]:
def getAllSeasonStandings(listOfSeasonLeague):
  listOfSeasonLeagueStandings = []

  for x in listOfSeasonLeague:
    teamSeasonLeagueJson = json.loads(x)
    jsonLeagueSeasonStandingsResp = getSeasonStandings(teamSeasonLeagueJson["season"],teamSeasonLeagueJson["league"])
    
    listOfSeasonLeagueStandings.append(str(jsonLeagueSeasonStandingsResp["response"]))
  saveJson("standing", "allSeasonLeagueStanding", listOfSeasonLeagueStandings)
  return listOfSeasonLeagueStandings

In [ ]:
#listOfSeasonLeague = getAllSeasonLeagueInformation(jsonLeagues)
#listOfSeasonLeagueStandings = getAllSeasonStandings(listOfSeasonLeague)
listOfSeasonLeagueStandings = importJson("standing/allSeasonLeagueStanding.json")

# Fixtures

**Update Frequency** : This endpoint is updated every 15 seconds.

**Recommended Calls** : 1 call per minute for the leagues, teams, fixtures who have at least one fixture in progress otherwise 1 call per day.

https://www.api-football.com/documentation-v3#tag/Fixtures

In [ ]:
def getFixturesLive(leagueId):
  fixturesRoundsLiveUrl = f"fixtures?live=all&league={str(leagueId)}"
  jsonFixturesLive = getApiSports(fixturesRoundsLiveUrl)
  return jsonFixturesLive

In [ ]:
def getFixtures(season, leagueId):
  fixturesRoundsUrl = f"fixtures?season={str(season)}&league={str(leagueId)}"
  jsonFixtures = getApiSports(fixturesRoundsUrl)
  path = f"fixtures/league/{str(leagueId)}/{str(season)}"
  filename = f"{str(leagueId)}_{str(season)}_fixtures"
  saveJson(path, filename, jsonFixtures)
  return jsonFixtures

In [ ]:
def getAllFixtures(listOfSeasonLeague):
  listOfSeasonLeagueFixtures = []

  for x in listOfSeasonLeague:
    fixtureSeasonLeagueJson = json.loads(x)
    jsonSeasonLeagueFixturesResp = getFixtures(fixtureSeasonLeagueJson["season"],fixtureSeasonLeagueJson["league"])
    for y in jsonSeasonLeagueFixturesResp["response"]:
      jsonSeasonLeagueFixtures = "{\"fixture\":" + str(y["fixture"]["id"]) + " ,\"league\": " + str(y["league"]["id"]) + " ,\"season\": " + str(y["league"]["season"]) + " ,\"team1\": " + str(y["teams"]["home"]["id"]) + " ,\"team2\": " + str(y["teams"]["away"]["id"]) + " ,\"date\": \"" + str(y["fixture"]["date"][:len(y["fixture"]["date"])-15]) + "\"}"
      listOfSeasonLeagueFixtures.append(str(jsonSeasonLeagueFixtures))
  
  saveJson("fixtures", "allSeasonLeagueFixtures", listOfSeasonLeagueFixtures)
  return listOfSeasonLeagueFixtures

In [ ]:
#listOfSeasonLeagueFixtures = getAllFixtures(listOfSeasonLeague)
listOfSeasonLeague = getAllSeasonLeagueInformation(jsonLeagues)
listOfSeasonLeagueFixtures = importJson("fixtures/allSeasonLeagueFixtures.json")

In [ ]:
#jsonFixturesLive = getFixturesLive(169) #liga chinesa teste

## Fixtures H2H
Podemos ir buscar o historico (mas esta api será mais usada no momento do confronto entre equipas do dia seguinte por exemplo)

**Update Frequency** : This endpoint is updated every 15 seconds.

**Recommended Calls** : 1 call per minute for the leagues, teams, fixtures who have at least one fixture in progress otherwise 1 call per day.

https://www.api-football.com/documentation-v3#tag/Fixtures/operation/get-fixtures-headtohead

In [ ]:
def getFixturesH2H(teamId1, teamId2, fromDate, toDate):
  fixtureH2HUrl = "fixtures/headtohead?h2h=" + str(teamId1) + "-" + str(teamId2) + "&from=" + fromDate + "&to=" + toDate
  jsonFixturesH2H = getApiSports(fixtureH2HUrl)
  return jsonFixturesH2H

def getFixturesH2HSaveJson(fixtureId, leagueId, season, teamId1, teamId2, fromDate, toDate):
  jsonFixturesH2HAux = getFixturesH2H(teamId1, teamId2, fromDate, toDate)
  path = "fixtures/league/" + str(leagueId) + "/" + str(season) + "/" + str(fixtureId)
  filename = str(leagueId) + "_" + str(season) + "_" + str(fixtureId) + "_fixtures_h2h"
  saveJson(path, filename, jsonFixturesH2HAux)
  return jsonFixturesH2HAux

In [ ]:
def getAllFixturesH2H(listOfSeasonLeagueFixtures, fromDate):
  listOfSeasonLeagueFixturesH2H = []
  for x in listOfSeasonLeagueFixtures:
    fixtureSeasonLeagueJson = json.loads(x) 
    if parser.parse(fixtureSeasonLeagueJson["date"]).date() <= date.today():
      jsonSeasonLeagueFixturesH2HResp = getFixturesH2HSaveJson(fixtureSeasonLeagueJson["fixture"],fixtureSeasonLeagueJson["league"],fixtureSeasonLeagueJson["season"],fixtureSeasonLeagueJson["team1"],fixtureSeasonLeagueJson["team2"], fromDate, fixtureSeasonLeagueJson["date"])
      #listOfSeasonLeagueFixturesH2H.append(jsonSeasonLeagueFixturesH2HResp) #demasiadas fixtures para ficarem guardadas numa variável
  return listOfSeasonLeagueFixtures

In [ ]:
#getFixturesH2H(417,194,"2011-12-11","2011-12-11") #as datas from e to são inclusives
#getAllFixturesH2H(listOfSeasonLeagueFixtures,"2000-01-01")

## Fixtures Statistics

**Update Frequency** : This endpoint is updated every minute.

**Recommended Calls** : 1 call every minute for the teams or fixtures who have at least one fixture in progress otherwise 1 call per day.

https://www.api-football.com/documentation-v3#tag/Fixtures/operation/get-fixtures-statistics

In [ ]:
def getFixturesStatistics(fixtureId):
  fixtureRoundsStatisticsUrl = "fixtures/statistics?fixture=" + str(fixtureId)
  jsonFixturesStatistics = getApiSports(fixtureRoundsStatisticsUrl)
  return jsonFixturesStatistics

def getFixturesStatisticsSaveJson(fixtureId, leagueId, season):
  jsonFixturesStatisticsAux = getFixturesStatistics(fixtureId)
  path = "fixtures/league/" + str(leagueId) + "/" + str(season) + "/" + str(fixtureId)
  filename = str(leagueId) + "_" + str(season) + "_" + str(fixtureId) + "_fixtures_statistics"
  saveJson(path, filename, jsonFixturesStatisticsAux)
  return jsonFixturesStatisticsAux

In [ ]:
def getAllFixturesStatistics(listOfSeasonLeagueFixtures):
  listOfSeasonLeagueFixturesStatistics = []
  for x in listOfSeasonLeagueFixtures:
    fixtureSeasonLeagueJson = json.loads(x) 
    if parser.parse(fixtureSeasonLeagueJson["date"]).date() <= date.today():
      jsonSeasonLeagueFixturesStatisticsResp = getFixturesStatisticsSaveJson(fixtureSeasonLeagueJson["fixture"],fixtureSeasonLeagueJson["league"],fixtureSeasonLeagueJson["season"])
      #listOfSeasonLeagueFixturesStatistics.append(jsonSeasonLeagueFixturesStatisticsResp) #demasiadas fixtures para ficarem guardadas numa variável
  return listOfSeasonLeagueFixturesStatistics

In [ ]:
#lstLeagues = [2,3]
#lstSeasons = [2021,2022] 
#lstSeasons = [2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]
#listOfSeasonLeagueFixturesFiltered = [x for x in listOfSeasonLeagueFixtures if json.loads(x)["league"] in lstLeagues and json.loads(x)["season"] in lstSeasons]
#getAllFixturesStatistics(listOfSeasonLeagueFixturesFiltered)

## Fixtures Events

VAR events are available from the 2020-2021 season.

**Update Frequency** : This endpoint is updated every 15 seconds.

**Recommended Calls** : 1 call per minute for the fixtures in progress otherwise 1 call per day.

https://www.api-football.com/documentation-v3#tag/Fixtures/operation/get-fixtures-events

In [ ]:
def getFixturesEvents(fixtureId):
  fixtureRoundsEventsUrl = "fixtures/events?fixture=" + str(fixtureId)
  jsonFixturesEvents = getApiSports(fixtureRoundsEventsUrl)
  return jsonFixturesEvents

def getFixturesEventsSaveJson(fixtureId, leagueId, season):
  jsonFixturesEventsAux = getFixturesEvents(fixtureId)
  path = "fixtures/league/" + str(leagueId) + "/" + str(season) + "/" + str(fixtureId)
  filename = str(leagueId) + "_" + str(season) + "_" + str(fixtureId) + "_fixtures_events"
  saveJson(path, filename, jsonFixturesEventsAux)
  return jsonFixturesEventsAux

In [ ]:
def getAllFixturesEvents(listOfSeasonLeagueFixtures):
  listOfSeasonLeagueFixturesEvents = []
  for x in listOfSeasonLeagueFixtures:
    fixtureSeasonLeagueJson = json.loads(x) 
    if parser.parse(fixtureSeasonLeagueJson["date"]).date() <= date.today():
      jsonSeasonLeagueFixturesEventsResp = getFixturesEventsSaveJson(fixtureSeasonLeagueJson["fixture"],fixtureSeasonLeagueJson["league"],fixtureSeasonLeagueJson["season"])
      #listOfSeasonLeagueFixturesEvents.append(jsonSeasonLeagueFixturesEventsResp) #demasiadas fixtures para ficarem guardadas numa variável
  return listOfSeasonLeagueFixturesEvents

In [ ]:
#lstLeagues = [78,203,848]
#lstSeasons = [2016,2017,2018,2019,2020,2021,2022] 
#lstSeasons = [2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]
#listOfSeasonLeagueFixturesFiltered = [x for x in listOfSeasonLeagueFixtures if json.loads(x)["league"] in lstLeagues and json.loads(x)["season"] in lstSeasons]
#getAllFixturesEvents(listOfSeasonLeagueFixturesFiltered)

## Fixtures Lineups

**Update Frequency** : This endpoint is updated every 15 minutes.

**Recommended Calls** : 1 call every 15 minutes for the fixtures in progress otherwise 1 call per day.

https://www.api-football.com/documentation-v3#tag/Fixtures/operation/get-fixtures-lineups

In [ ]:
def getFixturesLineups(fixtureId):
  fixtureRoundsLineupsUrl = "fixtures/lineups?fixture=" + str(fixtureId)
  jsonFixturesLineups = getApiSports(fixtureRoundsLineupsUrl)
  return jsonFixturesLineups

def getFixturesLineupsSaveJson(fixtureId, leagueId, season):
  jsonFixturesLineupsAux = getFixturesLineups(fixtureId)
  path = "fixtures/league/" + str(leagueId) + "/" + str(season) + "/" + str(fixtureId)
  filename = str(leagueId) + "_" + str(season) + "_" + str(fixtureId) + "_fixtures_lineups"
  saveJson(path, filename, jsonFixturesLineupsAux)
  return jsonFixturesLineupsAux

In [ ]:
def getAllFixturesLineups(listOfSeasonLeagueFixtures):
  listOfSeasonLeagueFixturesLineups = []
  for x in listOfSeasonLeagueFixtures:
    fixtureSeasonLeagueJson = json.loads(x) 
    if parser.parse(fixtureSeasonLeagueJson["date"]).date() <= date.today():
      jsonSeasonLeagueFixturesLineupsResp = getFixturesLineupsSaveJson(fixtureSeasonLeagueJson["fixture"],fixtureSeasonLeagueJson["league"],fixtureSeasonLeagueJson["season"])
      #listOfSeasonLeagueFixturesLineups.append(jsonSeasonLeagueFixturesLineupsResp) #demasiadas fixtures para ficarem guardadas numa variável
  return listOfSeasonLeagueFixturesLineups

In [ ]:
#lstLeagues = [135,88]
#lstSeasons = [2016,2017,2018,2019,2020,2021,2022] 
#lstSeasons = [2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]
#listOfSeasonLeagueFixturesFiltered = [x for x in listOfSeasonLeagueFixtures if json.loads(x)["league"] in lstLeagues and json.loads(x)["season"] in lstSeasons]
#getAllFixturesLineups(listOfSeasonLeagueFixturesFiltered)

## Fixtures Players Statistics

Get the players statistics from one fixture.

**Update Frequency** : This endpoint is updated every minute.

**Recommended Calls** : 1 call every minute for the fixtures in progress otherwise 1 call per day.

https://www.api-football.com/documentation-v3#tag/Fixtures/operation/get-fixtures-players

In [ ]:
def getFixturesPlayersStatistics(fixtureId):
  fixtureRoundsPlayersStatisticsUrl = "fixtures/players?fixture=" + str(fixtureId)
  jsonFixturesPlayersStatistics = getApiSports(fixtureRoundsPlayersStatisticsUrl)
  return jsonFixturesPlayersStatistics

def getFixturesPlayersStatisticsSaveJson(fixtureId, leagueId, season):
  jsonFixturesPlayersStatisticsAux = getFixturesPlayersStatistics(fixtureId)
  path = "fixtures/league/" + str(leagueId) + "/" + str(season) + "/" + str(fixtureId)
  filename = str(leagueId) + "_" + str(season) + "_" + str(fixtureId) + "_fixtures_players_statistics"
  saveJson(path, filename, jsonFixturesPlayersStatisticsAux)
  return jsonFixturesPlayersStatisticsAux

In [ ]:
def getAllFixturesPlayersStatistics(listOfSeasonLeagueFixtures):
  listOfSeasonLeagueFixturesPlayersStatistics = []
  for x in listOfSeasonLeagueFixtures:
    fixtureSeasonLeagueJson = json.loads(x) 
    if parser.parse(fixtureSeasonLeagueJson["date"]).date() <= date.today():
      jsonSeasonLeagueFixturesPlayersStatisticsResp = getFixturesPlayersStatisticsSaveJson(fixtureSeasonLeagueJson["fixture"],fixtureSeasonLeagueJson["league"],fixtureSeasonLeagueJson["season"])
      #listOfSeasonLeagueFixturesPlayersStatistics.append(jsonSeasonLeagueFixturesPlayersStatisticsResp) #demasiadas fixtures para ficarem guardadas numa variável
  return listOfSeasonLeagueFixturesPlayersStatistics

In [ ]:
#timeToSleep = 0 #esta api demora algum tempo a guardar o json logo não é necessário colocar um tempo de espera
#lstLeagues = [88]
#lstSeasons = [2019,2020,2021,2022] 
#lstSeasons = [2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]
#listOfSeasonLeagueFixturesFiltered = [x for x in listOfSeasonLeagueFixtures if json.loads(x)["league"] in lstLeagues and json.loads(x)["season"] in lstSeasons]
#getAllFixturesPlayersStatistics(listOfSeasonLeagueFixturesFiltered)

## Fixtures Predictions

**Update Frequency** : This endpoint is updated every hour.

**Recommended Calls** : 1 call per hour for the fixtures in progress otherwise 1 call per day.

https://www.api-football.com/documentation-v3#tag/Predictions/operation/get-predictions

In [ ]:
def getFixturesPredictions(fixtureId):
  fixtureRoundsPredictionsUrl = "predictions?fixture=" + str(fixtureId)
  jsonFixturesPredictions = getApiSports(fixtureRoundsPredictionsUrl)
  return jsonFixturesPredictions

def getFixturesPredictionsSaveJson(fixtureId, leagueId, season):
  jsonFixturesPredictionsAux = getFixturesPredictions(fixtureId)
  path = "fixtures/league/" + str(leagueId) + "/" + str(season) + "/" + str(fixtureId)
  filename = str(leagueId) + "_" + str(season) + "_" + str(fixtureId) + "_fixtures_predictions"
  saveJson(path, filename, jsonFixturesPredictionsAux)
  return jsonFixturesPredictionsAux

In [ ]:
def getAllFixturesPredictions(listOfSeasonLeagueFixtures):
  listOfSeasonLeagueFixturesPredictions = []
  for x in listOfSeasonLeagueFixtures:
    fixtureSeasonLeagueJson = json.loads(x) 
    if parser.parse(fixtureSeasonLeagueJson["date"]).date() <= date.today():
      jsonSeasonLeagueFixturesPredictionsResp = getFixturesPredictionsSaveJson(fixtureSeasonLeagueJson["fixture"],fixtureSeasonLeagueJson["league"],fixtureSeasonLeagueJson["season"])
      #listOfSeasonLeagueFixturesPredictions.append(jsonSeasonLeagueFixturesPredictionsResp) #demasiadas fixtures para ficarem guardadas numa variável
  return listOfSeasonLeagueFixturesPredictions

In [ ]:
#timeToSleep = 0.02 #esta api demora algum tempo a guardar o json logo não é necessário colocar um tempo de espera
#lstLeagues = [39,135,88,94]
#lstSeasons = [2019,2020,2021,2022] 
#lstSeasons = [2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]
#listOfSeasonLeagueFixturesFiltered = [x for x in listOfSeasonLeagueFixtures if json.loads(x)["league"] in lstLeagues and json.loads(x)["season"] in lstSeasons]
#getAllFixturesPredictions(listOfSeasonLeagueFixturesFiltered)

## Fixtures Injuries

**This endpoint requires at least one parameter.**

**Update Frequency** : This endpoint is updated every 4 hours.

**Recommended Calls** : 1 call per day.

https://www.api-football.com/documentation-v3#tag/Injuries/operation/get-injuries

In [ ]:
def getFixturesInjuries(fixtureId):
  fixtureRoundsInjuriesUrl = "injuries?fixture=" + str(fixtureId)
  jsonFixturesInjuries = getApiSports(fixtureRoundsInjuriesUrl)
  return jsonFixturesInjuries

def getFixturesInjuriesSaveJson(fixtureId, leagueId, season):
  jsonFixturesInjuriesAux = getFixturesInjuries(fixtureId)
  path = "fixtures/league/" + str(leagueId) + "/" + str(season) + "/" + str(fixtureId)
  filename = str(leagueId) + "_" + str(season) + "_" + str(fixtureId) + "_fixtures_injuries"
  saveJson(path, filename, jsonFixturesInjuriesAux)
  return jsonFixturesInjuriesAux

In [ ]:
def getAllFixturesInjuries(listOfSeasonLeagueFixtures):
  listOfSeasonLeagueFixturesInjuries = []
  for x in listOfSeasonLeagueFixtures:
    fixtureSeasonLeagueJson = json.loads(x) 
    if parser.parse(fixtureSeasonLeagueJson["date"]).date() <= date.today():
      jsonSeasonLeagueFixturesInjuriesResp = getFixturesInjuriesSaveJson(fixtureSeasonLeagueJson["fixture"],fixtureSeasonLeagueJson["league"],fixtureSeasonLeagueJson["season"])
      #listOfSeasonLeagueFixturesInjuries.append(jsonSeasonLeagueFixturesInjuriesResp) #demasiadas fixtures para ficarem guardadas numa variável
  return listOfSeasonLeagueFixturesInjuries

In [ ]:
#timeToSleep = 0.04 #esta api demora algum tempo a guardar o json logo não é necessário colocar um tempo de espera
#lstLeagues = [61,39,135,88,94]
#lstSeasons = [2019,2020,2021,2022] 
#lstSeasons = [2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]
#listOfSeasonLeagueFixturesFiltered = [x for x in listOfSeasonLeagueFixtures if json.loads(x)["league"] in lstLeagues and json.loads(x)["season"] in lstSeasons]
#getAllFixturesInjuries(listOfSeasonLeagueFixturesFiltered)

# Coach

**Update Frequency** : This endpoint is updated every day.

**Recommended Calls** : 1 call per day.

https://www.api-football.com/documentation-v3#tag/Coachs

In [ ]:
def getCoachsToFile(leagueId,season,team):
  coachTeamUrl = f"coachs?team={str(team)}"
  jsonCoachs = getApiSports(coachTeamUrl)
  path = f"coachs/league/{str(leagueId)}/{str(season)}/{str(team)}"
  filename = f"{str(leagueId)}_{str(season)}_{str(team)}_coachs"
  #jsonToCsv("response", "coachs", "coachs", jsonCoachs)
  saveJson(path, filename, jsonCoachs)
  return jsonCoachs

In [ ]:
def getAllSeasonLeagueCoachs(listOfSeasonLeagueTeams):
  listOfSeasonLeagueCoachs = []
  for x in listOfSeasonLeagueTeams:
    teamSeasonJson = json.loads(x)
    jsonLeagueSeasonCoachResp = getCoachsToFile(teamSeasonJson["league"],teamSeasonJson["season"],teamSeasonJson["team"])
    for y in jsonLeagueSeasonCoachResp["response"]:
      jsonLeagueSeasonCoach = "{\"coach\":" + str(y["id"]) + " ,\"league\": " + str(teamSeasonJson["league"]) + " ,\"season\": " + str(teamSeasonJson["season"]) + " ,\"team\": " + str(teamSeasonJson["team"]) + "}"
      listOfSeasonLeagueCoachs.append(str(jsonLeagueSeasonCoach))
  saveJson("coachs", "allSeasonLeagueCoachs", listOfSeasonLeagueCoachs)
  return listOfSeasonLeagueCoachs

In [ ]:
#jsonSeasonLeagueCoachs = getAllSeasonLeagueCoachs(listOfSeasonLeagueTeams)
jsonSeasonLeagueCoachs = importJson("coachs/allSeasonLeagueCoachs.json")

In [ ]:
listCoachsIdUnique = set()
for x in jsonSeasonLeagueCoachs:
  seasonLeagueCoachJson = json.loads(x)
  listCoachsIdUnique.add(str(seasonLeagueCoachJson["coach"]))
#print(len(listCoachsIdUnique))
listCoachsIdUnique = list(listCoachsIdUnique)

## Coach Sidelined

**Update Frequency** : This endpoint is updated several times a week.

**Recommended Calls** : 1 call per day.

https://www.api-football.com/documentation-v3#tag/Sidelined/operation/get-sidelined

In [ ]:
def getCoachSidelined(coach):
  coachSidelinedUrl = f"sidelined?coach={str(coach)}"
  jsonCoachs = getApiSports(coachSidelinedUrl)
  return jsonCoachs

In [ ]:
def saveAllSeasonLeagueCoachSidelined(jsonCoachs,jsonSeasonLeagueCoachs, coachId):
  jsonSeasonLeagueCoachsFiltered = [x for x in jsonSeasonLeagueCoachs if json.loads(x)["coach"] == int(coachId)]
  for x in jsonSeasonLeagueCoachsFiltered:
    teamSeasonJson = json.loads(x)
    leagueId = teamSeasonJson["league"]
    season = teamSeasonJson["season"]
    team = teamSeasonJson["team"]
    path = f"coachs/league/{str(leagueId)}/{str(season)}/{str(team)}"
    filename = f"{str(leagueId)}_{str(season)}_{str(team)}_coachs_sidelined"
    saveJson(path, filename, jsonCoachs)

In [ ]:
def getAllSeasonLeagueCoachsSidelined(listCoachsIdUnique,jsonSeasonLeagueCoachs):
  for x in listCoachsIdUnique:
    jsonCoachSidelined = getCoachSidelined(x)
    saveAllSeasonLeagueCoachSidelined(jsonCoachSidelined,jsonSeasonLeagueCoachs, x)

In [ ]:
#getAllSeasonLeagueCoachsSidelined(listCoachsIdUnique,jsonSeasonLeagueCoachs)

## Coach Trophies

**Update Frequency** : This endpoint is updated several times a week.

**Recommended Calls** : 1 call per day.

https://www.api-football.com/documentation-v3#tag/Trophies/operation/get-trophies

In [ ]:
def getCoachTrophies(coach):
  coachTrophiesUrl = f"trophies?coach={str(coach)}"
  jsonCoachs = getApiSports(coachTrophiesUrl)
  return jsonCoachs

In [ ]:
def saveAllSeasonLeagueCoachTrophies(jsonCoachs,jsonSeasonLeagueCoachs, coachId):
  jsonSeasonLeagueCoachsFiltered = [x for x in jsonSeasonLeagueCoachs if json.loads(x)["coach"] == int(coachId)]
  for x in jsonSeasonLeagueCoachsFiltered:
    teamSeasonJson = json.loads(x)
    leagueId = teamSeasonJson["league"]
    season = teamSeasonJson["season"]
    team = teamSeasonJson["team"]
    path = f"coachs/league/{str(leagueId)}/{str(season)}/{str(team)}"
    filename = f"{str(leagueId)}_{str(season)}_{str(team)}_coachs_trophies"
    saveJson(path, filename, jsonCoachs)

In [ ]:
def getAllSeasonLeagueCoachsTrophies(listCoachsIdUnique,jsonSeasonLeagueCoachs):
  for x in listCoachsIdUnique:
    jsonCoachTrophies = getCoachTrophies(x)
    saveAllSeasonLeagueCoachTrophies(jsonCoachTrophies,jsonSeasonLeagueCoachs, x)

In [ ]:
#getAllSeasonLeagueCoachsTrophies(listCoachsIdUnique,jsonSeasonLeagueCoachs)

## Coach Transfers

**Update Frequency** : This endpoint is updated several times a week.

**Recommended Calls** : 1 call per day.

https://www.api-football.com/documentation-v3#tag/Transfers/operation/get-transfers

In [ ]:
def getCoachTransfers(coach):
  coachTransfersUrl = f"transfers?coach={str(coach)}"
  jsonCoachs = getApiSports(coachTransfersUrl)
  return jsonCoachs

In [ ]:
def saveAllSeasonLeagueCoachTransfers(jsonCoachs,jsonSeasonLeagueCoachs, coachId):
  jsonSeasonLeagueCoachsFiltered = [x for x in jsonSeasonLeagueCoachs if json.loads(x)["coach"] == int(coachId)]
  for x in jsonSeasonLeagueCoachsFiltered:
    teamSeasonJson = json.loads(x)
    leagueId = teamSeasonJson["league"]
    season = teamSeasonJson["season"]
    team = teamSeasonJson["team"]
    path = f"coachs/league/{str(leagueId)}/{str(season)}/{str(team)}"
    filename = f"{str(leagueId)}_{str(season)}_{str(team)}_coachs_transfers"
    saveJson(path, filename, jsonCoachs)

In [ ]:
def getAllSeasonLeagueCoachsTransfers(listCoachsIdUnique,jsonSeasonLeagueCoachs):
  for x in listCoachsIdUnique:
    jsonCoachTransfers = getCoachTransfers(x)
    saveAllSeasonLeagueCoachTransfers(jsonCoachTransfers,jsonSeasonLeagueCoachs, x)

In [ ]:
#getAllSeasonLeagueCoachsTrophies(listCoachsIdUnique,jsonSeasonLeagueCoachs)

# Player
https://www.api-football.com/documentation-v3#tag/Players

## Player Season ( not implemented )

**Update Frequency** : This endpoint is updated every day.

**Recommended Calls** : 1 call per day.

https://www.api-football.com/documentation-v3#tag/Players/operation/get-players-seasons

In [ ]:
def getSeasonPlayersToFile():
  playerSeasonUrl = "players/seasons"
  jsonPlayers = getApiSports(playerSeasonUrl)
  path = "players"
  filename = "allSeasonPlayers"
  saveJson(path, filename, jsonPlayers)
  return jsonPlayers

In [ ]:
#jsonSeasonLeagueCoachs = getSeasonPlayersToFile()
#jsonSeasonLeagueCoachs = importJson("coachs/allSeasonPlayers.json")

## Player Statistics

**Update Frequency** : This endpoint is updated several times a week.

**Recommended Calls** : 1 call per day.

https://www.api-football.com/documentation-v3#tag/Coachs

In [ ]:
def getPlayersByTeamLeagueSeasonToFile(leagueId,season,team):
  page = 1 #first page
  totalPages = 1 #by default
  listJsonPlayers = []
  while page <= totalPages:
    playersByTeamLeagueSeasonUrl = f"players?team={str(team)}&season={str(season)}&league={str(leagueId)}&page={page}"
    jsonPlayers = getApiSports(playersByTeamLeagueSeasonUrl)
    path = f"players/league/{str(leagueId)}/{str(season)}/{str(team)}"
    filename = f"{str(leagueId)}_{str(season)}_{str(team)}_players_{str(page)}"
    saveJson(path, filename, jsonPlayers)
    listJsonPlayers.append(jsonPlayers)
    page += 1
    totalPages = jsonPlayers["paging"]["total"]
  return listJsonPlayers

In [ ]:
def getAllPlayersByTeamLeagueSeason(listOfSeasonLeagueTeams):
  listOfSeasonLeaguePlayers = []
  for x in listOfSeasonLeagueTeams:
    teamSeasonJson = json.loads(x)
    jsonLeagueSeasonPlayerResp = getPlayersByTeamLeagueSeasonToFile(teamSeasonJson["league"],teamSeasonJson["season"],teamSeasonJson["team"])
    for z in jsonLeagueSeasonPlayerResp:
      for y in z["response"]:
        jsonLeagueSeasonPlayer = "{\"player\":" + str(y["player"]["id"]) + " ,\"league\": " + str(teamSeasonJson["league"]) + " ,\"season\": " + str(teamSeasonJson["season"]) + " ,\"team\": " + str(teamSeasonJson["team"]) + "}"
        listOfSeasonLeaguePlayers.append(str(jsonLeagueSeasonPlayer))
  saveJson("players", "allSeasonLeaguePlayers", listOfSeasonLeaguePlayers)
  return listOfSeasonLeaguePlayers

In [ ]:
#lstLeagues = [39]
#lstSeasons = [2020] 
#listOfSeasonLeagueTeamsFiltered = [x for x in listOfSeasonLeagueTeams if json.loads(x)["league"] in lstLeagues and json.loads(x)["season"] in lstSeasons]
#jsonSeasonLeaguePlayers = getAllPlayersByTeamLeagueSeason(listOfSeasonLeagueTeams)
jsonSeasonLeaguePlayers = importJson("players/allSeasonLeaguePlayers.json")

In [ ]:
listPlayersIdUnique = set()
for x in jsonSeasonLeaguePlayers:
  seasonLeaguePlayerJson = json.loads(x)
  listPlayersIdUnique.add(str(seasonLeaguePlayerJson["player"]))
#print(len(listPlayersIdUnique))
listPlayersIdUnique = list(listPlayersIdUnique)

## Player Top Assists

**Update Frequency** : This endpoint is updated several times a week.

**Recommended Calls** : 1 call per day.

https://www.api-football.com/documentation-v3#tag/Players/operation/get-players-topassists

In [ ]:
def getPlayersTopAssistsToFile(leagueId,season):
  playersTopAssistsUrl = f"players/topassists?season={str(season)}&league={str(leagueId)}"
  jsonPlayers = getApiSports(playersTopAssistsUrl)
  path = f"players/league/{str(leagueId)}/{str(season)}"
  filename = f"{str(leagueId)}_{str(season)}_players_top_assists"
  saveJson(path, filename, jsonPlayers)
  return jsonPlayers

In [ ]:
def getAllPlayersTopAssists(listOfSeasonLeague):
  listOfSeasonLeaguePlayers = []
  for x in listOfSeasonLeague:
    teamSeasonJson = json.loads(x)
    jsonLeagueSeasonPlayerResp = getPlayersTopAssistsToFile(teamSeasonJson["league"],teamSeasonJson["season"])
    for y in jsonLeagueSeasonPlayerResp["response"]:
      jsonLeagueSeasonPlayer = "{\"player\":" + str(y["player"]["id"]) + " ,\"league\": " + str(teamSeasonJson["league"]) + " ,\"season\": " + str(teamSeasonJson["season"]) + "}"
      listOfSeasonLeaguePlayers.append(str(jsonLeagueSeasonPlayer))
  saveJson("players", "allSeasonLeaguePlayersTopAssists", listOfSeasonLeaguePlayers)
  return listOfSeasonLeaguePlayers

In [ ]:
#jsonPlayersTopAssists = getAllPlayersTopAssists(listOfSeasonLeague)
jsonPlayersTopAssists = importJson("players/allSeasonLeaguePlayersTopAssists.json")

## Player Top Scorers

**Update Frequency** : This endpoint is updated several times a week.

**Recommended Calls** : 1 call per day.

https://www.api-football.com/documentation-v3#tag/Players/operation/get-players-topscorers

In [ ]:
def getPlayersTopScoresToFile(leagueId,season):
  playersTopScoresUrl = f"players/topscorers?season={str(season)}&league={str(leagueId)}"
  jsonPlayers = getApiSports(playersTopScoresUrl)
  path = f"players/league/{str(leagueId)}/{str(season)}"
  filename = f"{str(leagueId)}_{str(season)}_players_top_scores"
  saveJson(path, filename, jsonPlayers)
  return jsonPlayers

In [ ]:
def getAllPlayersTopScores(listOfSeasonLeague):
  listOfSeasonLeaguePlayers = []
  for x in listOfSeasonLeague:
    teamSeasonJson = json.loads(x)
    jsonLeagueSeasonPlayerResp = getPlayersTopScoresToFile(teamSeasonJson["league"],teamSeasonJson["season"])
    for y in jsonLeagueSeasonPlayerResp["response"]:
      jsonLeagueSeasonPlayer = "{\"player\":" + str(y["player"]["id"]) + " ,\"league\": " + str(teamSeasonJson["league"]) + " ,\"season\": " + str(teamSeasonJson["season"]) + "}"
      listOfSeasonLeaguePlayers.append(str(jsonLeagueSeasonPlayer))
  saveJson("players", "allSeasonLeaguePlayersTopScores", listOfSeasonLeaguePlayers)
  return listOfSeasonLeaguePlayers

In [ ]:
#jsonPlayersTopScores = getAllPlayersTopScores(listOfSeasonLeague)
jsonPlayersTopScores = importJson("players/allSeasonLeaguePlayersTopScores.json")

## Player Top Yellow Cards

**Update Frequency** : This endpoint is updated several times a week.

**Recommended Calls** : 1 call per day.

https://www.api-football.com/documentation-v3#tag/Players/operation/get-players-topyellowcards

In [ ]:
def getPlayersTopYellowCardsToFile(leagueId,season):
  playersTopYellowCardsUrl = f"players/topyellowcards?season={str(season)}&league={str(leagueId)}"
  jsonPlayers = getApiSports(playersTopYellowCardsUrl)
  path = f"players/league/{str(leagueId)}/{str(season)}"
  filename = f"{str(leagueId)}_{str(season)}_players_top_yellow_cards"
  saveJson(path, filename, jsonPlayers)
  return jsonPlayers

In [ ]:
def getAllPlayersTopYellowCards(listOfSeasonLeague):
  listOfSeasonLeaguePlayers = []
  for x in listOfSeasonLeague:
    teamSeasonJson = json.loads(x)
    jsonLeagueSeasonPlayerResp = getPlayersTopYellowCardsToFile(teamSeasonJson["league"],teamSeasonJson["season"])
    for y in jsonLeagueSeasonPlayerResp["response"]:
      jsonLeagueSeasonPlayer = "{\"player\":" + str(y["player"]["id"]) + " ,\"league\": " + str(teamSeasonJson["league"]) + " ,\"season\": " + str(teamSeasonJson["season"]) + "}"
      listOfSeasonLeaguePlayers.append(str(jsonLeagueSeasonPlayer))
  saveJson("players", "allSeasonLeaguePlayersTopYellowCards", listOfSeasonLeaguePlayers)
  return listOfSeasonLeaguePlayers

In [ ]:
#jsonPlayersTopYellowCards = getAllPlayersTopYellowCards(listOfSeasonLeague)
jsonPlayersTopYellowCards = importJson("players/allSeasonLeaguePlayersTopYellowCards.json")

## Player Top Ref Cards

**Update Frequency** : This endpoint is updated several times a week.

**Recommended Calls** : 1 call per day.

https://www.api-football.com/documentation-v3#tag/Players/operation/get-players-topredcards

In [ ]:
def getPlayersTopRedCardsToFile(leagueId,season):
  playersTopRedCardsUrl = f"players/topredcards?season={str(season)}&league={str(leagueId)}"
  jsonPlayers = getApiSports(playersTopRedCardsUrl)
  path = f"players/league/{str(leagueId)}/{str(season)}"
  filename = f"{str(leagueId)}_{str(season)}_players_top_red_cards"
  saveJson(path, filename, jsonPlayers)
  return jsonPlayers

In [ ]:
def getAllPlayersTopRedCards(listOfSeasonLeague):
  listOfSeasonLeaguePlayers = []
  for x in listOfSeasonLeague:
    teamSeasonJson = json.loads(x)
    jsonLeagueSeasonPlayerResp = getPlayersTopRedCardsToFile(teamSeasonJson["league"],teamSeasonJson["season"])
    for y in jsonLeagueSeasonPlayerResp["response"]:
      jsonLeagueSeasonPlayer = "{\"player\":" + str(y["player"]["id"]) + " ,\"league\": " + str(teamSeasonJson["league"]) + " ,\"season\": " + str(teamSeasonJson["season"]) + "}"
      listOfSeasonLeaguePlayers.append(str(jsonLeagueSeasonPlayer))
  saveJson("players", "allSeasonLeaguePlayersTopRedCards", listOfSeasonLeaguePlayers)
  return listOfSeasonLeaguePlayers

In [ ]:
#jsonPlayersTopRedCards = getAllPlayersTopRedCards(listOfSeasonLeague)
jsonPlayersTopRedCards = importJson("players/allSeasonLeaguePlayersTopRedCards.json")

## Player Sidelined

**Update Frequency** : This endpoint is updated several times a week.

**Recommended Calls** : 1 call per day.

https://www.api-football.com/documentation-v3#tag/Sidelined/operation/get-sidelined

In [ ]:
def getPlayerSidelined(player):
  playerSidelinedUrl = f"sidelined?player={str(player)}"
  jsonPlayerSidelined = getApiSports(playerSidelinedUrl)
  return jsonPlayerSidelined

In [ ]:
def saveAllSeasonLeaguePlayerSidelined(jsonPlayers,jsonSeasonLeaguePlayers, playerId):
  jsonSeasonLeaguePlayersFiltered = [x for x in jsonSeasonLeaguePlayers if json.loads(x)["player"] == int(playerId)]
  for x in jsonSeasonLeaguePlayersFiltered:
    teamSeasonJson = json.loads(x)
    leagueId = teamSeasonJson["league"]
    season = teamSeasonJson["season"]
    team = teamSeasonJson["team"]
    path = f"players/league/{str(leagueId)}/{str(season)}/{str(team)}"
    filename = f"{str(leagueId)}_{str(season)}_{str(team)}_{str(playerId)}_players_sidelined"
    saveJson(path, filename, jsonPlayers)

In [ ]:
def getAllSeasonLeaguePlayersSidelined(listPlayersIdUnique,jsonSeasonLeaguePlayers):
  listPlayersIdUniqueFiltered = [x for x in listPlayersIdUnique if json.loads(x)["season"] == 2022]
  for x in listPlayersIdUniqueFiltered:
    jsonPlayerSidelined = getPlayerSidelined(x)
    saveAllSeasonLeaguePlayerSidelined(jsonPlayerSidelined,jsonSeasonLeaguePlayers, x)

In [ ]:
#getAllSeasonLeaguePlayersSidelined(listPlayersIdUnique,jsonSeasonLeaguePlayers)

## Player Trophies

**Update Frequency** : This endpoint is updated several times a week.

**Recommended Calls** : 1 call per day.

https://www.api-football.com/documentation-v3#tag/Sidelined/operation/get-sidelined

In [ ]:
def getPlayerTrophies(player):
  playerTrophiesUrl = f"trophies?player={str(player)}"
  jsonPlayerTrophies = getApiSports(playerTrophiesUrl)
  return jsonPlayerTrophies

In [ ]:
def saveAllSeasonLeaguePlayerTrophies(jsonPlayers,jsonSeasonLeaguePlayers, playerId):
  jsonSeasonLeaguePlayersFiltered = [x for x in jsonSeasonLeaguePlayers if json.loads(x)["player"] == int(playerId)]
  for x in jsonSeasonLeaguePlayersFiltered:
    teamSeasonJson = json.loads(x)
    leagueId = teamSeasonJson["league"]
    season = teamSeasonJson["season"]
    team = teamSeasonJson["team"]
    path = f"players/league/{str(leagueId)}/{str(season)}/{str(team)}"
    filename = f"{str(leagueId)}_{str(season)}_{str(team)}_{str(playerId)}_players_trophies"
    saveJson(path, filename, jsonPlayers)

In [ ]:
def getAllSeasonLeaguePlayersTrophies(listPlayersIdUnique,jsonSeasonLeaguePlayers):
  for x in listPlayersIdUnique:
    jsonPlayerTrophies = getPlayerTrophies(x)
    saveAllSeasonLeaguePlayerTrophies(jsonPlayerTrophies,jsonSeasonLeaguePlayers, x)

In [ ]:
#getAllSeasonLeaguePlayersTrophies(listPlayersIdUnique,jsonSeasonLeaguePlayers)
#for x in jsonSeasonLeaguePlayers:
#  teamSeasonJson = json.loads(x)
#  leagueId = teamSeasonJson["league"]
#  season = teamSeasonJson["season"]
#  team = teamSeasonJson["team"]
#  player = teamSeasonJson["player"]
#  path = f"players/league/{str(leagueId)}/{str(season)}/{str(team)}"
#  filename = f"{str(leagueId)}_{str(season)}_{str(team)}_{str(player)}_players_trophies.json"
#  fullPath = f"{filePath}/{path}/{filename}"
#  if season == 2022:
#  #if os.path.exists(fullPath) == False:
#    print(player)
#    listPlayersIdUniqueAux = [x for x in listPlayersIdUnique if x == str(player)]
#    getAllSeasonLeaguePlayersTrophies(listPlayersIdUniqueAux,jsonSeasonLeaguePlayers)

## Player Transfers

**Update Frequency** : This endpoint is updated several times a week.

**Recommended Calls** : 1 call per day.

https://www.api-football.com/documentation-v3#tag/Transfers/operation/get-transfers

In [ ]:
def getPlayerTransfers(player):
  playerTransfersUrl = f"transfers?player={str(player)}"
  jsonPlayers = getApiSports(playerTransfersUrl)
  return jsonPlayers

In [ ]:
def saveAllSeasonLeaguePlayerTransfers(jsonPlayers,jsonSeasonLeaguePlayers, player):
  jsonSeasonLeaguePlayersFiltered = [x for x in jsonSeasonLeaguePlayers if json.loads(x)["player"] == int(player)]
  for x in jsonSeasonLeaguePlayersFiltered:
    teamSeasonJson = json.loads(x)
    leagueId = teamSeasonJson["league"]
    season = teamSeasonJson["season"]
    team = teamSeasonJson["team"]
    path = f"players/league/{str(leagueId)}/{str(season)}/{str(team)}"
    filename = f"{str(leagueId)}_{str(season)}_{str(team)}_{str(player)}_players_transfers"
    saveJson(path, filename, jsonPlayers)

In [ ]:
def getAllSeasonLeaguePlayersTransfers(listPlayersIdUnique,jsonSeasonLeaguePlayers):
  for x in listPlayersIdUnique:
    jsonPlayerTransfers = getPlayerTransfers(x)
    saveAllSeasonLeaguePlayerTransfers(jsonPlayerTransfers,jsonSeasonLeaguePlayers, x)

In [ ]:
#getAllSeasonLeaguePlayersTransfers(listPlayersIdUnique,jsonSeasonLeaguePlayers)
#for x in jsonSeasonLeaguePlayers:
#  teamSeasonJson = json.loads(x)
#  leagueId = teamSeasonJson["league"]
#  season = teamSeasonJson["season"]
#  team = teamSeasonJson["team"]
#  player = teamSeasonJson["player"]
#  path = f"players/league/{str(leagueId)}/{str(season)}/{str(team)}"
#  filename = f"{str(leagueId)}_{str(season)}_{str(team)}_{str(player)}_players_transfers.json"
#  fullPath = f"{filePath}/{path}/{filename}"
#  if season == 2022:
#  #if os.path.exists(fullPath) == False:
#    print(player)
#    listPlayersIdUniqueAux = [x for x in listPlayersIdUnique if x == str(player)]
#    getAllSeasonLeaguePlayersTransfers(listPlayersIdUniqueAux,jsonSeasonLeaguePlayers)

# Tratamento dos dados

## Equipas

In [ ]:
leagueidfilter = 203
filename = f"{filePath}/fixtures/league/203/2011/509613/203_2011_509613_fixtures_predictions.json"

#f"{filePath}/fixtures/league/848/2021/714875/848_2021_714875_fixtures_predictions.json"
#f"{filePath}/fixtures/league/39/2011/193817/39_2011_193817_fixtures_predictions.json"
#f"{filePath}/fixtures/league/203/2011/509613/203_2011_509613_fixtures_predictions.json"
# Opening JSON file
f = open(filename)

# returns JSON object as 
# a dictionary
data = json.load(f)

#df = pd.read_json(filename)
#listOfSeasonLeagueFixtures

In [ ]:
#data["response"][0]

In [ ]:
fixturePrediction = pd.json_normalize(data["response"],max_level=6)
fixtureId = data["parameters"]["fixture"]
fixturePrediction["fixtureId"] = fixtureId
#fixturePrediction

In [ ]:
fixtureClean = fixturePrediction[['fixtureId', 'league.id', 'league.name', 'teams.home.id', 'teams.home.name','teams.away.id', 'teams.away.name','predictions.winner.id', 'predictions.win_or_draw', 'predictions.under_over',
 'predictions.goals.home', 'predictions.goals.away', 'predictions.percent.home', 'predictions.percent.draw', 'predictions.percent.away', 'predictions.advice',
 'teams.home.last_5.form', 'teams.home.last_5.att', 'teams.home.last_5.def', 'teams.home.last_5.goals.for.average', 'teams.home.last_5.goals.against.average',
 'teams.away.last_5.form', 'teams.away.last_5.att', 'teams.away.last_5.def', 'teams.away.last_5.goals.for.average', 'teams.away.last_5.goals.against.average',
 'teams.home.league.form', 'teams.home.league.fixtures.wins.home', 'teams.home.league.fixtures.wins.away', 
 'teams.home.league.fixtures.draws.home', 'teams.home.league.fixtures.draws.away','teams.home.league.fixtures.loses.home', 'teams.home.league.fixtures.loses.away',
 'teams.home.league.goals.for.average.home', 'teams.home.league.goals.for.average.away',
 'teams.away.league.form', 'teams.away.league.fixtures.wins.home', 'teams.away.league.fixtures.wins.away', 
 'teams.away.league.fixtures.draws.home', 'teams.away.league.fixtures.draws.away','teams.away.league.fixtures.loses.home', 'teams.away.league.fixtures.loses.away',
 'teams.away.league.goals.for.average.home', 'teams.away.league.goals.for.average.away'
 ]]

In [ ]:
#fixtureClean

In [ ]:
def importPredictionsDataFromJsonFiles(league,season,fixture,df_fixturePredictions):
  filename = f"{filePath}/fixtures/league/{str(league)}/{str(season)}/{str(fixture)}/{str(league)}_{str(season)}_{str(fixture)}_fixtures_predictions.json"

  # Opening JSON file
  f = open(filename)

  # returns JSON object as a dictionary
  data = json.load(f)
  fixturePrediction = pd.json_normalize(data["response"],max_level=6)
  fixturePrediction["fixtureId"] = data["parameters"]["fixture"]
  fixtureClean = fixturePrediction[['fixtureId', 'league.id', 'league.name','league.season', 'teams.home.id', 'teams.home.name','teams.away.id', 'teams.away.name','predictions.winner.id', 'predictions.win_or_draw', 'predictions.under_over',
      'predictions.goals.home', 'predictions.goals.away', 'predictions.percent.home', 'predictions.percent.draw', 'predictions.percent.away', 'predictions.advice',
      'teams.home.last_5.form', 'teams.home.last_5.att', 'teams.home.last_5.def', 'teams.home.last_5.goals.for.average', 'teams.home.last_5.goals.against.average',
      'teams.away.last_5.form', 'teams.away.last_5.att', 'teams.away.last_5.def', 'teams.away.last_5.goals.for.average', 'teams.away.last_5.goals.against.average',
      'teams.home.league.form', 'teams.home.league.fixtures.wins.home', 'teams.home.league.fixtures.wins.away', 
      'teams.home.league.fixtures.draws.home', 'teams.home.league.fixtures.draws.away','teams.home.league.fixtures.loses.home', 'teams.home.league.fixtures.loses.away',
      'teams.home.league.goals.for.average.home', 'teams.home.league.goals.for.average.away',
      'teams.away.league.form', 'teams.away.league.fixtures.wins.home', 'teams.away.league.fixtures.wins.away', 
      'teams.away.league.fixtures.draws.home', 'teams.away.league.fixtures.draws.away','teams.away.league.fixtures.loses.home', 'teams.away.league.fixtures.loses.away',
      'teams.away.league.goals.for.average.home', 'teams.away.league.goals.for.average.away'
      ]]
  df_fixturePredictions = df_fixturePredictions.append(fixtureClean, ignore_index=True)
  return df_fixturePredictions

In [ ]:
df_fixturePredictions = pd.DataFrame(fixtureClean)
listOfSeasonLeagueFixturesLeague2 = [x for x in listOfSeasonLeagueFixtures if json.loads(x)["league"] == int(leagueidfilter) and json.loads(x)["season"] < 2022]
for x in listOfSeasonLeagueFixturesLeague2:
  jsonLoad = json.loads(x)
  df_fixturePredictions = importPredictionsDataFromJsonFiles(str(jsonLoad["league"]),str(jsonLoad["season"]),str(jsonLoad["fixture"]),df_fixturePredictions)
df_fixturePredictions = df_fixturePredictions.drop(0)

In [ ]:
def importFixtureH2HDataFromJsonFiles(league,season,fixture,df_fixtureh2h,x):
  filename = f"{filePath}/fixtures/league/{str(league)}/{str(season)}/{str(fixture)}/{str(league)}_{str(season)}_{str(fixture)}_fixtures_h2h.json"

  # Opening JSON file
  f = open(filename)

  # returns JSON object as a dictionary
  data = json.load(f)
  fixtureh2h = pd.json_normalize(data["response"],max_level=6)
  fixtureClean = fixtureh2h[['fixture.id','fixture.date', 'score.halftime.home', 'score.halftime.away',
                             'score.fulltime.home', 'score.fulltime.away',
                             'score.extratime.home', 'score.extratime.away',
                             'score.penalty.home', 'score.penalty.away'
      ]]
  fixtureCleanFiltered = fixtureClean.loc[fixtureClean['fixture.id'] == int(fixture)]
  df_fixtureh2h['score.halftime.home'][x+1] = fixtureCleanFiltered['score.halftime.home'].values[0]
  df_fixtureh2h['score.halftime.away'][x+1] = fixtureCleanFiltered['score.halftime.away'].values[0]
  df_fixtureh2h['score.fulltime.home'][x+1] = fixtureCleanFiltered['score.fulltime.home'].values[0]
  df_fixtureh2h['score.fulltime.away'][x+1] = fixtureCleanFiltered['score.fulltime.away'].values[0]
  df_fixtureh2h['score.extratime.home'][x+1] = fixtureCleanFiltered['score.extratime.home'].values[0]
  df_fixtureh2h['score.extratime.away'][x+1] = fixtureCleanFiltered['score.extratime.away'].values[0]
  df_fixtureh2h['score.penalty.home'][x+1] = fixtureCleanFiltered['score.penalty.home'].values[0]
  df_fixtureh2h['score.penalty.away'][x+1] = fixtureCleanFiltered['score.penalty.away'].values[0]
  df_fixtureh2h['fixture.date'][x+1] = fixtureCleanFiltered['fixture.date'].values[0][0:10]
  return df_fixtureh2h

In [ ]:
l = [None] * len(df_fixturePredictions)
df_fixturePredictions["score.halftime.home"] = l
df_fixturePredictions["score.halftime.away"] = l
df_fixturePredictions["score.fulltime.home"] = l
df_fixturePredictions["score.fulltime.away"] = l
df_fixturePredictions["score.extratime.home"] = l
df_fixturePredictions["score.extratime.away"] = l
df_fixturePredictions["score.penalty.home"] = l
df_fixturePredictions["score.penalty.away"] = l
df_fixturePredictions["fixture.date"] = l

In [ ]:
df_fixturePredictionsh2h = df_fixturePredictions
for x in range(0, len(df_fixturePredictions)):
#  #print(df_fixturePredictions["league.id"].values[x])
  df_fixturePredictionsh2h = importFixtureH2HDataFromJsonFiles(df_fixturePredictions["league.id"].values[x], int(df_fixturePredictions["league.season"].values[x]), df_fixturePredictions["fixtureId"].values[x],df_fixturePredictionsh2h,x)

In [ ]:
df_fixturePredictionsh2h['predictions.percent.home'] = df_fixturePredictionsh2h['predictions.percent.home'].str.rstrip("%").astype(float)/100
df_fixturePredictionsh2h['predictions.percent.draw'] = df_fixturePredictionsh2h['predictions.percent.draw'].str.rstrip("%").astype(float)/100
df_fixturePredictionsh2h['predictions.percent.away'] = df_fixturePredictionsh2h['predictions.percent.away'].str.rstrip("%").astype(float)/100
df_fixturePredictionsh2h['teams.home.last_5.form'] = df_fixturePredictionsh2h['teams.home.last_5.form'].str.rstrip("%").astype(float)/100
df_fixturePredictionsh2h['teams.home.last_5.att'] = df_fixturePredictionsh2h['teams.home.last_5.att'].str.rstrip("%").astype(float)/100
df_fixturePredictionsh2h['teams.home.last_5.def'] = df_fixturePredictionsh2h['teams.home.last_5.def'].str.rstrip("%").astype(float)/100
df_fixturePredictionsh2h['teams.away.last_5.form'] = df_fixturePredictionsh2h['teams.away.last_5.form'].str.rstrip("%").astype(float)/100
df_fixturePredictionsh2h['teams.away.last_5.att'] = df_fixturePredictionsh2h['teams.away.last_5.att'].str.rstrip("%").astype(float)/100
df_fixturePredictionsh2h['teams.away.last_5.def'] = df_fixturePredictionsh2h['teams.away.last_5.def'].str.rstrip("%").astype(float)/100

In [ ]:
df_fixturePredictionsh2h = df_fixturePredictionsh2h.astype({"league.season": int})

In [ ]:
import numpy as np
df_fixturePredictionsh2h['predictions.win_or_draw'] = np.where(df_fixturePredictionsh2h['predictions.win_or_draw'] == "False", 0, np.where(df_fixturePredictionsh2h['predictions.win_or_draw'] == "True", 1, df_fixturePredictionsh2h['predictions.win_or_draw']))
df_fixturePredictionsh2h['result'] = np.where(df_fixturePredictionsh2h['score.fulltime.home'] == df_fixturePredictionsh2h['score.fulltime.away'], 0, np.where(df_fixturePredictionsh2h['score.fulltime.home'] < df_fixturePredictionsh2h['score.fulltime.away'], 0, np.where(df_fixturePredictionsh2h['score.fulltime.home'] > df_fixturePredictionsh2h['score.fulltime.away'], 1, df_fixturePredictionsh2h['score.fulltime.home'])))
#df_fixturePredictionsh2h['result'] = np.where(df_fixturePredictionsh2h['score.fulltime.home'] == df_fixturePredictionsh2h['score.fulltime.away'], 0, np.where(df_fixturePredictionsh2h['score.fulltime.home'] < df_fixturePredictionsh2h['score.fulltime.away'], 2, np.where(df_fixturePredictionsh2h['score.fulltime.home'] > df_fixturePredictionsh2h['score.fulltime.away'], 1, df_fixturePredictionsh2h['score.fulltime.home'])))

In [ ]:
df_fixtureCleanData =  df_fixturePredictionsh2h[['fixtureId', 'league.season', 'teams.home.id','teams.away.id', 
                                          #'predictions.winner.id', 'predictions.win_or_draw',
      #'predictions.percent.home', 'predictions.percent.draw', 'predictions.percent.away', 
      'teams.home.last_5.form', 'teams.home.last_5.att', 'teams.home.last_5.def', 'teams.home.last_5.goals.for.average', 'teams.home.last_5.goals.against.average',
      'teams.away.last_5.form', 'teams.away.last_5.att', 'teams.away.last_5.def', 'teams.away.last_5.goals.for.average', 'teams.away.last_5.goals.against.average',
      'teams.home.league.fixtures.wins.home', 'teams.home.league.fixtures.wins.away', 
      'teams.home.league.fixtures.draws.home', 'teams.home.league.fixtures.draws.away','teams.home.league.fixtures.loses.home', 'teams.home.league.fixtures.loses.away',
      'teams.home.league.goals.for.average.home', 'teams.home.league.goals.for.average.away',
      'teams.away.league.fixtures.wins.home', 'teams.away.league.fixtures.wins.away', 
      'teams.away.league.fixtures.draws.home', 'teams.away.league.fixtures.draws.away','teams.away.league.fixtures.loses.home', 'teams.away.league.fixtures.loses.away',
      'teams.away.league.goals.for.average.home', 'teams.away.league.goals.for.average.away',
      #, 'score.halftime.home', 'score.halftime.away',
      #  'score.fulltime.home', 'score.fulltime.away'
      'result'
      ]]

In [ ]:
df_fixtureCleanDataWithoutNan = df_fixtureCleanData.dropna(subset=['score.halftime.away', 'score.halftime.home','score.fulltime.away', 'score.fulltime.home'])

KeyError: ignored

In [ ]:
df_fixtureCleanData = df_fixtureCleanData.astype({"fixtureId": int,
                                                            "teams.home.last_5.goals.for.average": float,
                                                            "teams.home.last_5.goals.against.average": float,
                                                            "teams.away.last_5.goals.for.average": float,
                                                            "teams.away.last_5.goals.against.average": float
                                                            #"teams.home.league.goals.for.average.home": float,
                                                            #"teams.home.league.goals.for.average.away": float,
                                                            #"teams.away.league.goals.for.average.home": float,
                                                            #"teams.away.league.goals.for.average.away": float,
                                                            #"score.halftime.home": int,
                                                            #"score.halftime.away": int,
                                                            #"score.fulltime.home": int,
                                                            #"score.fulltime.away": int,
                                                            #"result": int
                                                            })

In [ ]:
matches = df_fixtureCleanData
matches = matches.reset_index()

In [ ]:
matches

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=50, min_samples_split=50, random_state=1)

In [ ]:
train = matches[matches["league.season"] < 2021]

In [ ]:
test = matches[matches["league.season"] >= 2021]

In [ ]:
predictors = ["teams.home.id", "teams.away.id"]

In [ ]:
rf.fit(train[predictors], train["result"])

In [ ]:
preds = rf.predict(test[predictors])

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
error = accuracy_score(test["result"], preds)

In [ ]:
error

In [ ]:
combined = pd.DataFrame(dict(actual=test["result"], predicted=preds))

In [ ]:
pd.crosstab(index=combined["actual"], columns=combined["predicted"])

In [ ]:
from sklearn.metrics import precision_score

precision_score(test["result"], preds)